# 행안부 도로명 주소 API 활용

In [2]:
import requests
from urllib.parse import quote # 한국어 검색어 사용할시 필요

API key 가져오기


In [3]:
from google.colab import files
up=files.upload()

Saving roadapikey.txt to roadapikey (1).txt


In [4]:
with open('roadapikey.txt') as rf:
  road_key=rf.read()

-url 만들기

In [5]:
#한글은 반드시 인코딩 해주어야 한다
quote('광주광역시청')

'%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C%EC%B2%AD'

In [39]:
#url은 copy-paste
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
parmas1 = f'currentPage=1&countPerPage=10&confmKey={road_key}'
params2 = f'keyword={quote("광주광역시 북구청")}&resultType=json'
url = f'{base_url}?{parmas1}&{params2}'

In [40]:
result = requests.get(url)
result.status_code

200

결과 해석하기 json-> python 데이터 구조로 변환

In [41]:
import json
res = json.loads(result.text)
res

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '3',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '77 Uchi-ro, Buk-gu, Gwangju',
    'rn': '우치로',
    'emdNm': '용봉동',
    'zipNo': '61187',
    'roadAddrPart2': '(용봉동)',
    'emdNo': '02',
    'sggNm': '북구',
    'jibunAddr': '광주광역시 북구 용봉동 239-2 북구청',
    'siNm': '광주광역시',
    'roadAddrPart1': '광주광역시 북구 우치로 77',
    'bdNm': '북구청',
    'admCd': '2917010700',
    'udrtYn': '0',
    'lnbrMnnm': '239',
    'roadAddr': '광주광역시 북구 우치로 77(용봉동)',
    'lnbrSlno': '2',
    'buldMnnm': '77',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '291703162056',
    'mtYn': '0',
    'bdMgtSn': '2917010700102390002024359',
    'buldSlno': '0'},
   {'detBdNmList': '한국청소년인권센터,노인복지센터',
    'engAddr': '86 Daecheon-ro, Buk-gu, Gwangju',
    'rn': '대천로',
    'emdNm': '문흥동',
    'zipNo': '61132',
    'roadAddrPart2': '(문흥동)',
    'emdNo': '01',
    'sggNm': '북구',
    'jibunAddr': '

In [37]:
juso=res.get('results').get('juso')
juso

[]

In [38]:
re_juso=juso[0].get('roadAddr')
re_juso

IndexError: ignored

In [30]:
re_re_juso=res.get('results').get('juso')[0].get('roadAddr')
re_re_juso

'광주광역시 서구 내방로 111(치평동)'

In [32]:
road_addr =res['results']['juso'][0]['roadAddr']
road_addr

'광주광역시 서구 내방로 111(치평동)'

-광주광역시 주요 행정기관 주소

In [44]:
places = '광주광역시청,광주광역시 동구청,광주광역시 북구청,광주광역시 남구청,광주광역시 서구청,광주광역시 광산구청'.split(',')
places

['광주광역시청', '광주광역시 동구청', '광주광역시 북구청', '광주광역시 남구청', '광주광역시 서구청', '광주광역시 광산구청']

In [47]:
addr_list=[]
for i in places:
  base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
  parmas1 = f'currentPage=1&countPerPage=10&confmKey={road_key}'
  params2 = f'keyword={quote(i)}&resultType=json'
  url = f'{base_url}?{parmas1}&{params2}'
  result = requests.get(url)
  res = json.loads(result.text)
  road_addr =res['results']['juso'][0]['roadAddr']
  addr_list.append(road_addr)

In [48]:
addr_list

['광주광역시 서구 내방로 111(치평동)',
 '광주광역시 동구 서남로 1(서석동)',
 '광주광역시 북구 우치로 77(용봉동)',
 '광주광역시 남구 봉선로 1(봉선동)',
 '광주광역시 서구 경열로 33(농성동)',
 '광주광역시 광산구 광산로29번길 15(송정동)']

- DaTaFrame으로 만들어 저장하기


In [49]:
import pandas as pd
df= pd.DataFrame({'장소':places,'도로명주소':addr_list})
df

,장소,도로명주소
0,광주광역시청,광주광역시 서구 내방로 111(치평동)
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동)
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동)
3,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동)
4,광주광역시 서구청,광주광역시 서구 경열로 33(농성동)
5,광주광역시 광산구청,광주광역시 광산구 광산로29번길 15(송정동)


In [50]:
df.to_csv('광주광역시내_주요기관.csv', index=False)